In [1]:
import os
total_images = 0
base_dir = r"/Users/fatimatuzzahra/Downloads/processed_slices/train"

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith('.png'):
            total_images += 1

print("Total images in testing folder:", total_images)


Total images in testing folder: 84825


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
import clip
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [3]:
base_dir = r"/Users/fatimatuzzahra/Downloads/processed_slices/train"
class_names = ['AD', 'CN', 'MCI']
total_images = 0

for class_name in class_names:
    path = os.path.join(base_dir, class_name, 'axial')
    if os.path.exists(path):
        num_files = len([f for f in os.listdir(path) if f.lower().endswith('.png')])
        print(f"{class_name} axial images: {num_files}")
        total_images += num_files
    else:
        print(f" Path not found: {path}")

print("Total axial images:", total_images)


AD axial images: 17575
CN axial images: 25795
MCI axial images: 41455
Total axial images: 84825


In [4]:
# Extracting CLIP Image features
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess_clip = clip.load("ViT-B/32", device=device)

import os
import numpy as np
from tqdm import tqdm
import imgaug.augmenters as iaa
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
# from tensorflow.keras.applications import DenseNet121.  # densenet code
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
import torchvision.transforms as T

# Extracting CLIP Image features
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess_clip = clip.load("ViT-B/32", device=device)

# Define your own compatible augmentations
augmenter = T.Compose([
    T.Resize(224),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(10),
    T.GaussianBlur(kernel_size=3),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))  # CLIP normalization
])

# EfficientNet base model
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
# # Step 3: Build feature extractor model (outputs 256-dim feature vectors)
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# feature_extractor = Model(inputs=base_model.input, outputs=x)

# Step 4: Setup paths and label mapping
base_dir = r"/Users/fatimatuzzahra/Downloads/processed_slices/train"
classes = ['AD', 'CN', 'MCI']
label_map = {cls: idx for idx, cls in enumerate(classes)}

features, labels, image_paths = [], [], []

# Step 5: Loop through dataset and extract features
for cls in classes:
    print(cls)
    class_dir = os.path.join(base_dir, cls, 'axial')
    if not os.path.exists(class_dir):
        print(f"Directory not found: {class_dir}")
        continue

    for fname in tqdm(os.listdir(class_dir), desc=f"Processing {cls}"):
        if fname.lower().endswith('.png'):
            img_path = os.path.join(class_dir, fname)
            try:
                pil_img = Image.open(img_path).convert("RGB")
                
                # --------- Original Image ---------
                orig_img = preprocess_clip(pil_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    orig_feat = model.encode_image(orig_img).cpu().numpy().flatten()
                features.append(orig_feat)
                labels.append(label_map[cls])
                image_paths.append(img_path)

                # --------- Augmented Image ---------
                aug_img_tensor = augmenter(pil_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    aug_feat = model.encode_image(aug_img_tensor).cpu().numpy().flatten()
                features.append(aug_feat)
                labels.append(label_map[cls])
                image_paths.append(img_path + "_aug")

            except Exception as e:
                print(f"Failed to process {img_path}: {e}")

# Step 6: Save features, labels, and paths
features = np.array(features)
labels = np.array(labels)
image_paths = np.array(image_paths)

np.save("train_features_axial.npy", features)
np.save("train_labels_axial.npy", labels)
np.save("train_image_paths_axial.npy", image_paths)

print("Feature vectors saved:")
print("train_features_axial.npy")
print("train_labels_axial.npy")
print("train_image_paths_axial.npy")

same process as above for validation data

import os
import numpy as np
from tqdm import tqdm
import imgaug.augmenters as iaa
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
# from tensorflow.keras.applications import DenseNet121.  # densenet code
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
import torchvision.transforms as T



# Define your own compatible augmentations
augmenter = T.Compose([
    T.Resize(224),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(10),
    T.GaussianBlur(kernel_size=3),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))  # CLIP normalization
])


# EfficientNet ase model
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
# # Step 3: Build feature extractor model (outputs 256-dim feature vectors)
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# feature_extractor = Model(inputs=base_model.input, outputs=x)

# Step 4: Setup paths and label mapping
base_dir = r"/Users/fatimatuzzahra/Downloads/processed_slices/val"
classes = ['AD', 'CN', 'MCI']
label_map = {cls: idx for idx, cls in enumerate(classes)}

features, labels, image_paths = [], [], []

# Step 5: Loop through dataset and extract features
for cls in classes:
    print(cls)
    class_dir = os.path.join(base_dir, cls, 'axial')
    if not os.path.exists(class_dir):
        print(f"Directory not found: {class_dir}")
        continue

    for fname in tqdm(os.listdir(class_dir), desc=f"Processing {cls}"):
        if fname.lower().endswith('.png'):
            img_path = os.path.join(class_dir, fname)
            try:
                img = preprocess_clip(Image.open(img_path)).unsqueeze(0).to(device)
                with torch.no_grad():
                    feat = model.encode_image(img)
                    feat = feat.cpu().numpy().flatten()
                features.append(feat)
                labels.append(label_map[cls])
                image_paths.append(img_path)
            except Exception as e:
                print(f"Failed: {img_path}, {e}")

# Step 6: Save features, labels, and paths
features = np.array(features)
labels = np.array(labels)
image_paths = np.array(image_paths)

np.save("val_features_axial.npy", features)
np.save("val_labels_axial.npy", labels)
np.save("val_image_paths_axial.npy", image_paths)

print("Feature vectors saved:")
print("val_features_axial.npy")
print("val_labels_axial.npy")
print("val_image_paths_axial.npy")

features = np.load("train_features_axial.npy")       # Shape: (84755, 256)
print(features.shape)
labels = np.load("train_labels_axial.npy")           # Shape: (84755,)
print(labels.shape)
image_paths = np.load("train_image_paths_axial.npy") # Shape: (84755,)
print(image_paths.shape)


loading and printing features and other things for validation data as well

features = np.load("val_features_axial.npy")       # Shape: (84755, 256)
print(features.shape)
labels = np.load("val_labels_axial.npy")           # Shape: (84755,)
print(labels.shape)
image_paths = np.load("val_image_paths_axial.npy") # Shape: (84755,)
print(image_paths.shape)


# Load the .npy files
import pandas as pd
features = np.load("train_features_axial.npy")       # Shape: (84755, 256)
labels = np.load("train_labels_axial.npy")           # Shape: (84755,)

# Combine features and labels
combined = np.hstack((features, labels.reshape(-1, 1)))  # Shape: (84755, 257)

# Create a DataFrame
df = pd.DataFrame(combined)

# Optionally name columns

feature_columns = [f"f{i}" for i in range(features.shape[1])]
df.columns = feature_columns + ["label"]

# Save to CSV
df.to_csv("train_axial_features_and_labels_only.csv", index=False)

print(" Saved: train_axial_features_and_labels_only.csv")


same process for validation set

# Load the .npy files
features = np.load("val_features_axial.npy")       # Shape: (84755, 256)
labels = np.load("val_labels_axial.npy")           # Shape: (84755,)

# Combine features and labels
combined = np.hstack((features, labels.reshape(-1, 1)))  # Shape: (84755, 257)

# Create a DataFrame
df = pd.DataFrame(combined)

# Optionally name columns
feature_columns = [f"f{i}" for i in range(features.shape[1])]
df.columns = feature_columns + ["label"]

# Save to CSV
df.to_csv("val_axial_features_and_labels_only.csv", index=False)

print(" Saved: val_axial_features_and_labels_only.csv")


In [3]:
# for text embeddings
import pandas as pd

# Load your file
df = pd.read_csv("/Users/fatimatuzzahra/Downloads/ADNI1_Complete_1Yr_1.5T_12_20_2024.csv")

# Preview the first few rows
print(df.shape)
df.head()


(2294, 12)


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I112538,941_S_1311,MCI,M,70,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,6/01/2008,NiFTI,12/07/2024
1,I97341,941_S_1311,MCI,M,70,m06,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,9/27/2007,NiFTI,12/07/2024
2,I97327,941_S_1311,MCI,M,69,sc,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/02/2007,NiFTI,12/07/2024
3,I63874,941_S_1202,CN,M,78,sc,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,1/30/2007,NiFTI,12/07/2024
4,I75150,941_S_1202,CN,M,78,m06,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/24/2007,NiFTI,12/07/2024


In [4]:
import pandas as pd
import numpy as np
import torch
import clip

# Load CSV
df = pd.read_csv("/Users/fatimatuzzahra/Downloads/ADNI1_Complete_1Yr_1.5T_12_20_2024.csv")

# 🔹 Step 1: Keep label and encode it numerically
label_map = {'AD': 0, 'CN': 1, 'MCI': 2}
df = df[df['Group'].isin(label_map)]  # filter out unknowns
df['label'] = df['Group'].map(label_map)

# 🔹 Step 2: Prepare text input (excluding label columns)
text_input_cols = [col for col in df.columns if col not in ['Group', 'label', 'Downloaded', 'Modality', 'Type', 'Format']]
df_text = df[text_input_cols]

# 🔹 Optional: Keep Image IDs if you want to track
image_ids = df['Image Data ID'].values

# 🔹 Step 3: Combine row text into string per patient
texts = df_text.astype(str).agg(" ".join, axis=1).tolist()

# 🔹 Step 4: Tokenize with CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
text_tokens = clip.tokenize(texts).to(device)

# 🔹 Step 5: Generate text embeddings
with torch.no_grad():
    text_embeddings = model.encode_text(text_tokens).cpu().numpy()

# 🔹 Step 6: Save embeddings and labels (ready for training!)
np.save("train_text_features_only.npy", text_embeddings)
np.save("train_text_labels_only.npy", df['label'].values)

# Optional: save a CSV for review
df_out = pd.DataFrame(text_embeddings)
df_out['label'] = df['label'].values
df_out.to_csv("train_text_features_with_labels.csv", index=False)

print("Saved:")
print(" - train_text_features_only.npy", text_embeddings.shape)
print(" - train_text_labels_only.npy", df['label'].shape)

Saved:
 - train_text_features_only.npy (2294, 512)
 - train_text_labels_only.npy (2294,)


In [5]:
df = pd.read_csv("train_text_features_with_labels.csv")

# Preview the first few rows
print(df.shape)
df.head()

print("Final dataframe shape:", df.shape)
print("All column names:", df.columns.tolist())

(2294, 513)
Final dataframe shape: (2294, 513)
All column names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '

In [1]:
import numpy as np
import os

# Load image paths (original splits)
train_paths = np.load("train_image_paths_axial.npy", allow_pickle=True)
val_paths = np.load("val_image_paths_axial.npy", allow_pickle=True)
test_paths = np.load("test_image_paths_axial.npy", allow_pickle=True)

# Extract image IDs from filenames (e.g., "I12345_AD_axial_45.png" -> "I12345")
def extract_ids(paths):
    return set(os.path.basename(p).split("_")[0] for p in paths)

train_ids = extract_ids(train_paths)
val_ids = extract_ids(val_paths)
test_ids = extract_ids(test_paths)

# Load textual data
text_features = np.load("train_text_features_only.npy")  # shape: (2294, D)
text_labels = np.load("train_text_labels_only.npy")
text_ids = np.load("text_image_ids.npy", allow_pickle=True)

# Match and split text data
train_feats, val_feats, test_feats = [], [], []
train_labs, val_labs, test_labs = [], [], []

for feat, label, tid in zip(text_features, text_labels, text_ids):
    if tid in train_ids:
        train_feats.append(feat)
        train_labs.append(label)
    elif tid in val_ids:
        val_feats.append(feat)
        val_labs.append(label)
    elif tid in test_ids:
        test_feats.append(feat)
        test_labs.append(label)

# Convert to numpy arrays
train_feats, val_feats, test_feats = map(np.array, [train_feats, val_feats, test_feats])
train_labs, val_labs, test_labs = map(np.array, [train_labs, val_labs, test_labs])

# Save
np.save("text_train_features.npy", train_feats)
np.save("text_train_labels.npy", train_labs)
np.save("text_val_features.npy", val_feats)
np.save("text_val_labels.npy", val_labs)
np.save("text_test_features.npy", test_feats)
np.save("text_test_labels.npy", test_labs)

print("✅ Done!")
print("Train:", train_feats.shape)
print("Val:", val_feats.shape)
print("Test:", test_feats.shape)

✅ Done!
Train: (1541, 512)
Val: (153, 512)
Test: (226, 512)


In [7]:
import numpy as np
import os

# Load original image paths
train_paths = np.load("train_image_paths_axial.npy", allow_pickle=True)
val_paths = np.load("val_image_paths_axial.npy", allow_pickle=True)
test_paths = np.load("test_image_paths_axial.npy", allow_pickle=True)

# Extract image IDs (e.g., "I12345_AD_axial_55.png" → "I12345")
def extract_ids(paths):
    return set(os.path.basename(p).split("_")[0] for p in paths)

train_ids = extract_ids(train_paths)
val_ids = extract_ids(val_paths)
test_ids = extract_ids(test_paths)

# 1. Check for overlaps between splits
overlap_train_val = train_ids & val_ids
overlap_train_test = train_ids & test_ids
overlap_val_test = val_ids & test_ids

# Print results
print("🔍 Checking for ID overlaps between splits:")
print("Train ∩ Val:", len(overlap_train_val))
print("Train ∩ Test:", len(overlap_train_test))
print("Val ∩ Test:", len(overlap_val_test))

# 2. (Optional) Also check if any text_image_ids are used in multiple splits
text_ids = np.load("text_image_ids.npy", allow_pickle=True)
text_ids_set = set(text_ids)

text_in_train = text_ids_set & train_ids
text_in_val = text_ids_set & val_ids
text_in_test = text_ids_set & test_ids

print("\n📦 Text sample distribution:")
print("Text IDs in Train:", len(text_in_train))
print("Text IDs in Val:", len(text_in_val))
print("Text IDs in Test:", len(text_in_test))

# Sanity check for overlaps in textual split
print("\n🔁 Overlap in Textual Data:")
print("Text Train ∩ Val:", len(text_in_train & text_in_val))
print("Text Train ∩ Test:", len(text_in_train & text_in_test))
print("Text Val ∩ Test:", len(text_in_val & text_in_test))

🔍 Checking for ID overlaps between splits:
Train ∩ Val: 0
Train ∩ Test: 0
Val ∩ Test: 0

📦 Text sample distribution:
Text IDs in Train: 1541
Text IDs in Val: 153
Text IDs in Test: 226

🔁 Overlap in Textual Data:
Text Train ∩ Val: 0
Text Train ∩ Test: 0
Text Val ∩ Test: 0


import numpy as np
import pandas as pd

# Load the cleaned text embeddings and associated image IDs
text_embeddings = np.load("text_embeddings_cleaned.npy")           # shape: (2294, 384)
text_image_ids = np.load("text_image_ids.npy", allow_pickle=True)  # shape: (2294,)

# Convert to DataFrame and add IDs as the first column
df = pd.DataFrame(text_embeddings)
df.insert(0, "Image_ID", text_image_ids)

# Save to CSV
df.to_csv("text_embeddings_cleaned_with_ids.csv", index=False)

print(" Saved: text_embeddings_cleaned_with_ids.csv")
print(" Shape:", df.shape)


import numpy as np
from tqdm import tqdm
import os

# Load image feature data
image_features = np.load("train_features_axial.npy")       # (84755, 256)
image_labels = np.load("train_labels_axial.npy")           # (84755,)
image_paths = np.load("train_image_paths_axial.npy")       # (84755,)

# Load cleaned textual embeddings and IDs
textual_embeddings = np.load("text_embeddings_cleaned.npy")   # (2294, 384)
text_ids = np.load("text_image_ids.npy", allow_pickle=True)      # (2294,)

# Step 1: Build a lookup from Image ID → text embedding
text_lookup = {id_: emb for id_, emb in zip(text_ids, textual_embeddings)}

# Step 2: Match each image with its text embedding (based on ID prefix)
fused_features = []
fused_labels = []
matched_count = 0

for i, (img_feat, label, path) in enumerate(tqdm(zip(image_features, image_labels, image_paths), total=len(image_paths))):
    filename = os.path.basename(path)  # e.g., 'I31143_AD_axial_55.png'
    img_id = filename.split('_')[0]             # 'I31143'

    if img_id in text_lookup:
        text_feat = text_lookup[img_id]
        fused = np.concatenate([img_feat, text_feat])  # shape (640,)
        fused_features.append(fused)
        fused_labels.append(label)
        matched_count += 1
        

print(f" Matched samples: {matched_count}")

# Convert to arrays and save
fused_features = np.array(fused_features)
fused_labels = np.array(fused_labels)

np.save("train_fused_features_clean.npy", fused_features)
np.save("train_fused_labels_clean.npy", fused_labels)

print(" Final train fused shape:", fused_features.shape)
print(" Labels shape:", fused_labels.shape)


In [14]:
# Extract image IDs from image paths
image_ids = set(os.path.basename(p).split('_')[0] for p in image_paths)

# Convert text_ids to set
text_ids_set = set(text_ids)

# Compute how many text IDs have a matching image ID
matching_ids = text_ids_set & image_ids
print("✅ Matched text IDs:", len(matching_ids))
print("❌ Unmatched text IDs:", len(text_ids_set - image_ids))
print("Total text IDs:", len(text_ids_set))


✅ Matched text IDs: 1541
❌ Unmatched text IDs: 753
Total text IDs: 2294


fusion for the validation set is done below

import numpy as np
from tqdm import tqdm
import os

# Load image feature data
image_features = np.load("val_features_axial.npy")       # (84755, 256)
image_labels = np.load("val_labels_axial.npy")           # (84755,)
image_paths = np.load("val_image_paths_axial.npy")       # (84755,)

# Load cleaned textual embeddings and IDs
textual_embeddings = np.load("text_embeddings_cleaned.npy")   # (2294, 384)
text_ids = np.load("text_image_ids.npy", allow_pickle=True)      # (2294,)

# Step 1: Build a lookup from Image ID → text embedding
text_lookup = {id_: emb for id_, emb in zip(text_ids, textual_embeddings)}

# Step 2: Match each image with its text embedding (based on ID prefix)
fused_features = []
fused_labels = []
matched_count = 0

for i, (img_feat, label, path) in enumerate(tqdm(zip(image_features, image_labels, image_paths), total=len(image_paths))):
    filename = os.path.basename(path)  # e.g., 'I31143_AD_axial_55.png'
    img_id = filename.split('_')[0]             # 'I31143'

    if img_id in text_lookup:
        text_feat = text_lookup[img_id]
        fused = np.concatenate([img_feat, text_feat])  # shape (640,)
        fused_features.append(fused)
        fused_labels.append(label)
        matched_count += 1

print(f" Matched samples: {matched_count}")

# Convert to arrays and save
fused_features = np.array(fused_features)
fused_labels = np.array(fused_labels)

np.save("val_fused_features_clean.npy", fused_features)
np.save("val_fused_labels_clean.npy", fused_labels)

print(" Final val fused shape:", fused_features.shape)
print(" Labels shape:", fused_labels.shape)


import numpy as np
import pandas as pd

# Load
X = np.load("train_fused_features_clean.npy")
y = np.load("train_fused_labels_clean.npy")

# Combine into DataFrame
df = pd.DataFrame(X)
df["label"] = y

# Save to CSV
df.to_csv("train_fused_embeddings_with_labels.csv", index=False)
print(" Saved: train_fused_embeddings_with_labels.csv")


savinf into csv the validation fused sets

import numpy as np
import pandas as pd

# Load
X = np.load("val_fused_features_clean.npy")
y = np.load("val_fused_labels_clean.npy")

# Combine into DataFrame
df = pd.DataFrame(X)
df["label"] = y

# Save to CSV
df.to_csv("val_fused_embeddings_with_labels.csv", index=False)
print(" Saved: val_fused_embeddings_with_labels.csv")


saving image and textual data separately for training dataset

import numpy as np

# Load fused embeddings
fused = np.load("train_fused_features_clean.npy")  # (84755, 640)

# Split features
image_features = fused[:, :256]   # CNN-based
text_features  = fused[:, 256:]   # Sentence-transformer-based

# Save separately
np.save("train_image_features_only.npy", image_features)
np.save("train_text_features_only.npy", text_features)

print(" Saved:")
print("  train_image_features_only.npy (shape:", image_features.shape, ")")
print("  train_text_features_only.npy  (shape:", text_features.shape, ")")


saving image and textual data separately for validation dataset

import numpy as np

# Load fused embeddings
fused = np.load("val_fused_features_clean.npy")  # (84755, 640)

# Split features
image_features = fused[:, :256]   # CNN-based
text_features  = fused[:, 256:]   # Sentence-transformer-based

# Save separately
np.save("val_image_features_only.npy", image_features)
np.save("val_text_features_only.npy", text_features)

print(" Saved:")
print("  val_image_features_only.npy (shape:", image_features.shape, ")")
print("  val_text_features_only.npy  (shape:", text_features.shape, ")")


the code below is added just to save the progress

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 🔹 Step 1: Load training set fused features and labels
X_train = np.load("train_fused_features_clean.npy")   # Shape: (84755, 640)
y_train = np.load("train_fused_labels_clean.npy")     # Shape: (84755,)

# Step 2: Load validation set fused features and labels
X_val = np.load("val_fused_features_clean.npy")
y_val = np.load("val_fused_labels_clean.npy")

# 🔹 Step 2: Train/test split (not used for testing file)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# 🔹 Step 3: Define and train MLP (no need of fitting again in test set)
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 64),   # You can adjust the architecture
    activation='relu',
    learning_rate_init=0.0006,
    solver='adam',
    max_iter=50,          # Increase to 100–300 for better results if time allows
    random_state=42,
    verbose=True
)
mlp.fit(X_train, y_train)

# 🔹 Step 4: Predict and evaluate
y_pred = mlp.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"\n MLP Accuracy: {acc:.4f}")
print("\n Classification Report:")
print(classification_report(y_test, y_pred, target_names=["AD", "CN", "MCI"]))

# 🔹 Step 5: Plot Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=["AD", "CN", "MCI"], yticklabels=["AD", "CN", "MCI"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(" Confusion Matrix - MLP (Fused Features)")
plt.tight_layout()
plt.show()


In [5]:
import torch.nn as nn

# --------------------- Define MLP Model ---------------------
class MLPWithDropout(nn.Module):
    def __init__(self):
        super(MLPWithDropout, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 3)  # 3 classes: AD, CN, MCI
        )

    def forward(self, x):
        return self.net(x)


In [6]:
import pandas as pd
import numpy as np
import torch
import clip

# Load CSV
df = pd.read_csv("/Users/fatimatuzzahra/Downloads/ADNI1_Complete_1Yr_1.5T_12_20_2024.csv")

# 🔹 Step 1: Keep label and encode it numerically
label_map = {'AD': 0, 'CN': 1, 'MCI': 2}
df = df[df['Group'].isin(label_map)]  # filter out unknowns
df['label'] = df['Group'].map(label_map)

# 🔹 Step 2: Prepare text input (excluding label columns)
text_input_cols = [col for col in df.columns if col not in ['Group', 'label', 'Downloaded', 'Modality', 'Type', 'Format']]
df_text = df[text_input_cols]

# 🔹 Optional: Keep Image IDs if you want to track
image_ids = df['Image Data ID'].values

# 🔹 Step 3: Combine row text into string per patient
texts = df_text.astype(str).agg(" ".join, axis=1).tolist()

# 🔹 Step 4: Tokenize with CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
text_tokens = clip.tokenize(texts).to(device)

# 🔹 Step 5: Generate text embeddings
with torch.no_grad():
    text_embeddings = model.encode_text(text_tokens).cpu().numpy()

# 🔹 Step 6: Save embeddings and labels (ready for training!)
np.save("train_text_features_only.npy", text_embeddings)
np.save("train_text_labels_only.npy", df['label'].values)

# Optional: save a CSV for review
df_out = pd.DataFrame(text_embeddings)
df_out['label'] = df['label'].values
df_out.to_csv("train_text_features_with_labels.csv", index=False)

print("✅ Saved:")
print(" - train_text_features_only.npy", text_embeddings.shape)
print(" - train_text_labels_only.npy", df['label'].shape)

✅ Saved:
 - train_text_features_only.npy (2294, 512)
 - train_text_labels_only.npy (2294,)


In [35]:
import os

# Load the image split path files
train_paths = np.load("train_image_paths_axial.npy", allow_pickle=True)
val_paths = np.load("val_image_paths_axial.npy", allow_pickle=True)
test_paths = np.load("test_image_paths_axial.npy", allow_pickle=True)

def extract_ids(paths):
    return set(os.path.basename(p).split("_")[0] for p in paths)

train_ids = extract_ids(train_paths)
val_ids = extract_ids(val_paths)
test_ids = extract_ids(test_paths)

In [36]:
# Load your text embeddings and image IDs:

text_embeddings = np.load("train_text_features_only.npy")
text_labels = np.load("train_text_labels_only.npy")
text_ids = np.load("text_image_ids.npy", allow_pickle=True)  # assuming you saved these earlier
text_ids = df['Image Data ID'].values

In [37]:
# Now match and split

train_feats, val_feats, test_feats = [], [], []
train_labs, val_labs, test_labs = [], [], []

for emb, label, tid in zip(text_embeddings, text_labels, text_ids):
    tid_str = str(tid)
    if tid_str in train_ids:
        train_feats.append(emb)
        train_labs.append(label)
    elif tid_str in val_ids:
        val_feats.append(emb)
        val_labs.append(label)
    elif tid_str in test_ids:
        test_feats.append(emb)
        test_labs.append(label)

In [38]:
# Convert and save if needed (for cross-checking

train_feats, val_feats, test_feats = map(np.array, [train_feats, val_feats, test_feats])
train_labs, val_labs, test_labs = map(np.array, [train_labs, val_labs, test_labs])

np.save("text_train_features.npy", train_feats)
np.save("text_train_labels.npy", train_labs)

np.save("text_val_features.npy", val_feats)
np.save("text_val_labels.npy", val_labs)

np.save("text_test_features.npy", test_feats)
np.save("text_test_labels.npy", test_labs)

print("Train:", train_feats.shape)
print("Val:", val_feats.shape)
print("Test:", test_feats.shape)

Train: (1541, 512)
Val: (153, 512)
Test: (226, 512)


In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import gc

# --------------------- Load Data ---------------------
X_train = np.load("train_text_features_only.npy")
y_train = np.load("train_text_labels_only.npy")

X_val = np.load("text_val_features.npy")
y_val = np.load("text_val_labels.npy")

# --------------------- Normalize ---------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# --------------------- SMOTE Oversampling ---------------------
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

# --------------------- Tensor Conversion ---------------------
X_train_tensor = torch.tensor(X_train_bal, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_bal, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)


# --------------------- Training Setup ---------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MLPWithDropout().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 100


best_val_loss = float('inf')
epochs_no_improve = 0
patience = 5  # You can tune this
early_stop = False

best_model_state = None  # To save the best model
# --------------------- Train Loop ---------------------
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # ---------------- Validate ----------------
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # -------- Early Stopping Check --------
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"⛔ Early stopping at epoch {epoch+1}")
            early_stop = True
            break
gc.collect()
torch.cuda.empty_cache()

# --------------------- Evaluate on Training Set ---------------------
model.eval()
y_train_preds = []
y_train_true = []

with torch.no_grad():
    for xb, yb in train_loader:
        xb = xb.to(device)
        out = model(xb)
        preds = torch.argmax(out, dim=1)
        y_train_preds.extend(preds.cpu().numpy())
        y_train_true.extend(yb.numpy())

train_acc = accuracy_score(y_train_true, y_train_preds)
print(f"📊 Training Accuracy: {train_acc:.4f}")

# --------------------- Evaluate on Validation Set ---------------------
y_val_preds = []
y_val_true = []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        out = model(xb)
        preds = torch.argmax(out, dim=1)
        y_val_preds.extend(preds.cpu().numpy())
        y_val_true.extend(yb.numpy())

val_acc = accuracy_score(y_val_true, y_val_preds)
print(f"✅ Validation Accuracy: {val_acc:.4f}")
print("\nClassification Report:\n", classification_report(y_val_true, y_val_preds, target_names=["AD", "CN", "MCI"]))

# --------------------- Confusion Matrix ---------------------
cm = confusion_matrix(y_val_true, y_val_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Purples", xticklabels=["AD", "CN", "MCI"], yticklabels=["AD", "CN", "MCI"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - MLP (Validation Set)")
plt.tight_layout()
plt.show()

: 

In [40]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming y_test and y_pred are already defined from your previous evaluation

# Define class labels (you can update these if your label encoding is different)
class_names = ['AD', 'CN', 'MCI']

# Accuracy
accuracy = accuracy_score(y_val_true, y_val_preds)
print(f"\n Accuracy: {accuracy:.4f}")

# Precision, Recall, F1-score
print("\n Classification Report:")
print(classification_report(y_val_true, y_val_preds, target_names=class_names))



 Accuracy: 0.4314

 Classification Report:
              precision    recall  f1-score   support

          AD       0.23      0.24      0.23        34
          CN       0.42      0.38      0.40        47
         MCI       0.53      0.56      0.54        72

    accuracy                           0.43       153
   macro avg       0.39      0.39      0.39       153
weighted avg       0.43      0.43      0.43       153



	.   ✅ Replace np.concatenate() fusion with a trainable layer
	•	✅ Try BioCLIP or medical image-pretrained models
	•	✅ Improve tabular/text embedding via BioBERT or using key columns only
	•	✅ Regularize fusion with modality dropout
	•	✅ (Optional) Try late fusion/ensemble as a baseline